In [ ]:
#Казанцевой Дарьи_Школа DA

In [242]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelEncoder

In [243]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

/Users/nosocheeeek/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# АЭРОПОРТ С МИНИМАЛЬНОЙ ЗАДЕРЖКОЙ ВЫЛЕТА

In [244]:
#Проверим столбец ORIGIN_AIRPORT на наличие пропусков
flights.ORIGIN_AIRPORT.isna().sum()

0

In [245]:
#Посмотрим, какие аэропорты, из которых отправляются рейсы, есть в датафрейме
flights.ORIGIN_AIRPORT.unique().tolist()

['ANC',
 'LAX',
 'SFO',
 'SEA',
 'LAS',
 'DEN',
 'SLC',
 'PDX',
 'FAI',
 'MSP',
 'PHX',
 'SJU',
 'PBG',
 'IAG',
 'PSE',
 'BQN',
 'ORD',
 'GEG',
 'HNL',
 'ONT',
 'MCO',
 'BOS',
 'HIB',
 'ABR',
 'MAF',
 'DFW',
 'MKE',
 'IAH',
 'BNA',
 'BRO',
 'VPS',
 'BOI',
 'BJI',
 'SGF',
 'PHL',
 'SBN',
 'RDD',
 'EUG',
 'IAD',
 'BUF',
 'PWM',
 'JFK',
 'CRP',
 'PIA',
 'FAT',
 'SMF',
 'AUS',
 'MCI',
 'ATL',
 'JAX',
 'MFR',
 'IDA',
 'MSN',
 'DCA',
 'SAT',
 'CHS',
 'SBA',
 'SMX',
 'IND',
 'CLE',
 'GSP',
 'BDL',
 'ABI',
 'RIC',
 'BFL',
 'OMA',
 'RDM',
 'FLL',
 'CID',
 'TPA',
 'SYR',
 'ROC',
 'TYR',
 'LAN',
 'XNA',
 'GSO',
 'EWR',
 'PBI',
 'RSW',
 'OAK',
 'PVD',
 'RNO',
 'PIT',
 'ABQ',
 'MIA',
 'BWI',
 'LGA',
 'TUL',
 'LIT',
 'MSY',
 'OKC',
 'ATW',
 'PNS',
 'MEM',
 'TYS',
 'MHT',
 'SAV',
 'CLT',
 'GRB',
 'ABE',
 'JAN',
 'OAJ',
 'FAR',
 'ERI',
 'LEX',
 'CWA',
 'MSO',
 'TTN',
 'AMA',
 'CLL',
 'HOU',
 'JLN',
 'MLI',
 'RDU',
 'CVG',
 'MHK',
 'MOB',
 'TLH',
 'BHM',
 'CAE',
 'TXK',
 'ACY',
 'DTW',
 'RAP',
 'TUS',


In [246]:
flights.DESTINATION_AIRPORT.unique().tolist()

['SEA',
 'PBI',
 'CLT',
 'MIA',
 'ANC',
 'MSP',
 'DFW',
 'ATL',
 'IAH',
 'PDX',
 'MCI',
 'FLL',
 'ORD',
 'HNL',
 'PHX',
 'EWR',
 'JFK',
 'MCO',
 'BOS',
 'BDL',
 'ITO',
 'SFO',
 'KOA',
 'OGG',
 'MYR',
 'DTW',
 'LIH',
 'DEN',
 'SJU',
 'LAX',
 'BWI',
 'IAD',
 'BQN',
 'BUF',
 'LGA',
 'HOU',
 'SLC',
 'PHL',
 'SJC',
 'OAK',
 'LGB',
 'TPA',
 'DCA',
 'TTN',
 'BTR',
 'LAS',
 'RSW',
 'BRD',
 'STL',
 'RKS',
 'MBS',
 'SNA',
 'MEI',
 'MDW',
 'SAN',
 'RIC',
 'AUS',
 'OTZ',
 'PIT',
 'JAX',
 'MSY',
 'ONT',
 'PSP',
 'BUR',
 'DAL',
 'CVG',
 'SMF',
 'RDU',
 'JMS',
 'BNA',
 'DSM',
 'MAF',
 'BOI',
 'ELP',
 'TUS',
 'SCC',
 'HPN',
 'STT',
 'MDT',
 'RHI',
 'SBP',
 'MKE',
 'JNU',
 'CMH',
 'CLD',
 'KTN',
 'CAK',
 'CRP',
 'CLE',
 'GPT',
 'SHV',
 'TYS',
 'IND',
 'LIT',
 'SAT',
 'SRQ',
 'TUL',
 'GRK',
 'PNS',
 'BTV',
 'CHS',
 'DAY',
 'OKC',
 'SAV',
 'XNA',
 'COS',
 'GJT',
 'BZN',
 'PUB',
 'HRL',
 'HDN',
 'MEM',
 'GEG',
 'ORH',
 'SYR',
 'GSO',
 'VPS',
 'LAW',
 'ACY',
 'LBB',
 'JAC',
 'BIL',
 'EUG',
 'ASE',
 'TVC',


In [ ]:
#По выведенным данным несложно отметить, что не все аэропорты приведены в кодировке IATA.
#Нужно заменить пятизначные значения FAA в IATA, чтобы сделать данные однородными. 
#Для этого я нашла в Интернете два csv-файла, один содержит все аэропорты США в кодировке IATA, другой в FAA.

In [247]:
#Решаем проблему с несоответствующим кодом в колонке ORIGIN_AIRPORT
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

In [248]:
aircode1.head(5)

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [249]:
aircode2.head(5)

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


In [250]:
#Преобразуем датафреймы для дальнейшего использования
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

In [251]:
for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [252]:
#Для определения аэропорта с минимальной задержкой вылета нам не нужны несостоявшиеся вылеты
ncanceled_flights = flights.query('CANCELLED == False')

In [255]:
#Проверим столбец DEPARTURE_DELAY на наличие пропусков
ncanceled_flights.DEPARTURE_DELAY.isna().sum()

0

In [256]:
#Рассчитаем среднее время задержки вылета в каждом аэропорте
mean_delays = ncanceled_flights.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].mean()

In [257]:
mean_delays.sort_values()

ORIGIN_AIRPORT
YAK    -6.070932
CNY    -6.058537
EKO    -3.933333
VEL    -3.740000
CDV    -3.454545
         ...    
OTH    18.969697
PPG    20.844828
HYA    23.182927
MVY    24.685185
ILG    29.568421
Name: DEPARTURE_DELAY, Length: 323, dtype: float64

In [258]:
min_delay_airport = mean_delays.idxmin()

In [255]:
airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [259]:
min_delay_airport = airports.query('IATA_CODE == @min_delay_airport')

In [260]:
print(f'Аэропорт с минимальной задержкой вылета - {min_delay_airport.AIRPORT.iloc[0]}')

Аэропорт с минимальной задержкой вылета - Yakutat Airport


САМАЯ ПУНКТУАЛЬНАЯ АВИАКОМПАНИЯ НА ПРИЛЕТ В Los Angeles International Airport

In [ ]:
#В DESTINATION_AIRPORT также встречается кодировка FAA. Исправим.

In [261]:
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

In [262]:
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [263]:
la_airport = airports.query('AIRPORT == "Los Angeles International Airport"')['IATA_CODE'].iloc[0]

In [264]:
punc = flights.query('DESTINATION_AIRPORT == @la_airport')

In [269]:
punc = pd.DataFrame(punc)

In [265]:
punc

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
101,2015,1,1,4,OO,6512,N925SW,FAT,LAX,535,...,650.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
103,2015,1,1,4,OO,4986,N719SK,AUS,LAX,538,...,654.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
129,2015,1,1,4,OO,2599,N435SW,DEN,LAX,545,...,811.0,56.0,0,0,NaN,0.0,0.0,56.0,0.0,0.0
131,2015,1,1,4,OO,6457,N560SW,SMX,LAX,545,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
156,2015,1,1,4,UA,1222,N68452,MCO,LAX,555,...,840.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818936,2015,12,31,4,AA,14,N858AA,OGG,LAX,2246,...,653.0,55.0,0,0,NaN,0.0,0.0,24.0,31.0,0.0
5818976,2015,12,31,4,UA,1110,N76516,OGG,LAX,2310,...,549.0,-47.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819002,2015,12,31,4,AA,246,N863AA,KOA,LAX,2326,...,635.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819018,2015,12,31,4,UA,1229,N57869,HNL,LAX,2336,...,658.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
punc.ARRIVAL_DELAY.isna().sum()

2652

In [267]:
#В ARRIVAL_DELAY есть многочисленные пропуски. Для того, чтобы заполнить их, рассчитаем время задержки по данным
#в SCHEDULED_ARRIVAL и ARRIVAL_TIME

In [270]:
# Преобразуем временные метки в формат datetime
punc['SCHEDULED_ARRIVAL'] = pd.to_datetime(punc['SCHEDULED_ARRIVAL'])
punc['ARRIVAL_TIME'] = pd.to_datetime(punc['ARRIVAL_TIME'])

# Рассчитаем задержку в минутах
punc['ARRIVAL_DELAY'] = (punc['ARRIVAL_TIME'] - punc['SCHEDULED_ARRIVAL']).dt.total_seconds() / 60

In [271]:
#Если самолёт прилетел раньше запланированного, задержкой это считать не будем
punc.loc[punc['ARRIVAL_DELAY'] < 0, 'ARRIVAL_DELAY'] = 0

In [272]:
# Найдем среднюю задержку для каждой авиакомпании
avg_delays = punc.groupby('AIRLINE')['ARRIVAL_DELAY'].mean().reset_index()

# Сортируем по средней задержке и выбираем первую строку (самую пунктуальную авиакомпанию)
most_punctual = avg_delays.sort_values(by='ARRIVAL_DELAY').head(1)

print(f"Самая пунктуальная авиакомпания на прилет в LAX: {most_punctual['AIRLINE'].values[0]} с средней задержкой {most_punctual['ARRIVAL_DELAY'].values[0]:.2f} минут.")

Самая пунктуальная авиакомпания на прилет в LAX: HA с средней задержкой 0.00 минут.


Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [275]:
#Сделаем сводные таблицы по TAXI_OUT и TAXI_IN для каждого аэропорта, 
#затем объединим для высчитывания среднего времени на рулении для каждого аэропорта

t1 = flights.TAXI_IN.notna()
t2 = flights.TAXI_OUT.notna()

df = flights[t1&t2]

In [276]:
pivot_table1 = pd.pivot_table(
    df,
    index='ORIGIN_AIRPORT', 
    values=['TAXI_IN', 'FLIGHT_NUMBER'],  
    aggfunc={
        'TAXI_IN': 'sum',  
        'FLIGHT_NUMBER': 'count'  
    }
)

In [277]:
pivot_table1

,FLIGHT_NUMBER,TAXI_IN
ORIGIN_AIRPORT,,
ABE,2473,20767.0
ABI,2426,25815.0
ABQ,20776,159166.0
ABR,733,5685.0
ABY,954,7494.0
...,...,...
WRG,707,2942.0
WYS,208,1243.0
XNA,9920,113291.0


In [278]:
pivot_table2 = pd.pivot_table(
    df,
    index='DESTINATION_AIRPORT',  
    values=['TAXI_OUT', 'FLIGHT_NUMBER'], 
    aggfunc={
        'TAXI_OUT': 'sum',  
        'FLIGHT_NUMBER': 'count'  
    }
)

In [279]:
pivot_table2

,FLIGHT_NUMBER,TAXI_OUT
DESTINATION_AIRPORT,,
ABE,2461,46602.0
ABI,2421,36577.0
ABQ,20792,306556.0
ABR,733,13315.0
ABY,955,17157.0
...,...,...
WRG,713,7995.0
WYS,208,4200.0
XNA,9949,193582.0


In [280]:
pivot_table1 = pivot_table1.reset_index().astype(str)
pivot_table2 = pivot_table2.reset_index().astype(str)
pivot_table1 = pivot_table1.rename(columns={'ORIGIN_AIRPORT': 'AIRPORT'})
pivot_table2 = pivot_table2.rename(columns={'DESTINATION_AIRPORT': 'AIRPORT'})

pivot = pd.merge(pivot_table1,pivot_table2,on = 'AIRPORT')

In [281]:
pivot

,AIRPORT,FLIGHT_NUMBER_x,TAXI_IN,FLIGHT_NUMBER_y,TAXI_OUT
0,ABE,2473,20767.0,2461,46602.0
1,ABI,2426,25815.0,2421,36577.0
2,ABQ,20776,159166.0,20792,306556.0
3,ABR,733,5685.0,733,13315.0
4,ABY,954,7494.0,955,17157.0
...,...,...,...,...,...
318,WRG,707,2942.0,713,7995.0
319,WYS,208,1243.0,208,4200.0
320,XNA,9920,113291.0,9949,193582.0
321,YAK,712,2806.0,713,8329.0


In [282]:
pivot['FLIGHT_NUMBER_x'] = pivot.FLIGHT_NUMBER_x.astype(int)
pivot['FLIGHT_NUMBER_y'] = pivot.FLIGHT_NUMBER_y.astype(int)
pivot['TAXI_IN'] = pivot.TAXI_IN.astype(float)
pivot['TAXI_OUT'] = pivot.TAXI_OUT.astype(float)

In [283]:
pivot['MEAN_TOTAL_TAXI'] = (pivot['TAXI_IN']+pivot['TAXI_OUT'])/(pivot['FLIGHT_NUMBER_x']+pivot['FLIGHT_NUMBER_y'])

In [284]:
pivot

,AIRPORT,FLIGHT_NUMBER_x,TAXI_IN,FLIGHT_NUMBER_y,TAXI_OUT,MEAN_TOTAL_TAXI
0,ABE,2473,20767.0,2461,46602.0,13.654033
1,ABI,2426,25815.0,2421,36577.0,12.872292
2,ABQ,20776,159166.0,20792,306556.0,11.203859
3,ABR,733,5685.0,733,13315.0,12.960437
4,ABY,954,7494.0,955,17157.0,12.913043
...,...,...,...,...,...,...
318,WRG,707,2942.0,713,7995.0,7.702113
319,WYS,208,1243.0,208,4200.0,13.084135
320,XNA,9920,113291.0,9949,193582.0,15.444814
321,YAK,712,2806.0,713,8329.0,7.814035


In [285]:
pivot.MEAN_TOTAL_TAXI.isna().sum()

0

In [286]:
# Группировка по аэропорту назначения и расчёт среднего времени руления
airport_taxi_time = pivot.groupby('AIRPORT')['MEAN_TOTAL_TAXI'].mean()
airport_taxi_time

AIRPORT
ABE    13.654033
ABI    12.872292
ABQ    11.203859
ABR    12.960437
ABY    12.913043
         ...    
WRG     7.702113
WYS    13.084135
XNA    15.444814
YAK     7.814035
YUM    15.235858
Name: MEAN_TOTAL_TAXI, Length: 323, dtype: float64

In [287]:
# Поиск аэропорта с максимальным средним временем руления
max_taxi_airport = airport_taxi_time.idxmax()
max_taxi_time = airport_taxi_time.max()

# Вывод результатов
print(f'Аэропорт с наибольшим средним временем руления: {max_taxi_airport}')
print(f'Среднее время руления: {max_taxi_time:.2f} минут')

Аэропорт с наибольшим средним временем руления: MQT
Среднее время руления: 19.88 минут


Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета  

In [288]:
# Преобразуем временные метки в формат datetime
flights['SCHEDULED_ARRIVAL'] = pd.to_datetime(flights['SCHEDULED_ARRIVAL'])
flights['ARRIVAL_TIME'] = pd.to_datetime(flights['ARRIVAL_TIME'])

# Рассчитаем задержку в минутах
flights['ARRIVAL_DELAY'] = (flights['ARRIVAL_TIME'] - flights['SCHEDULED_ARRIVAL']).dt.total_seconds() / 60

In [289]:
flights[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','ARRIVAL_DELAY']]

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY
0,ANC,SEA,-3.666667e-10
1,LAX,PBI,-1.500000e-10
2,SFO,CLT,8.333333e-11
3,LAX,MIA,-8.166667e-10
4,SEA,ANC,-1.016667e-09
...,...,...,...
5819074,LAX,BOS,-1.100000e-09
5819075,JFK,PSE,-2.666667e-10
5819076,JFK,SJU,-1.333333e-10
5819077,MCO,SJU,-1.666667e-10


In [290]:
flights.ARRIVAL_DELAY.fillna(flights.ARRIVAL_DELAY.median(),inplace = True)

In [294]:
# Выбор необходимых столбцов
X = pd.DataFrame(flights[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']])
y = pd.DataFrame(flights['ARRIVAL_DELAY'])

In [295]:
# Кодирование категориальных признаков
le_origin = LabelEncoder()
le_dest = LabelEncoder()

X['ORIGIN_AIRPORT'] = le_origin.fit_transform(X['ORIGIN_AIRPORT'])
X['DESTINATION_AIRPORT'] = le_dest.fit_transform(X['DESTINATION_AIRPORT'])

In [296]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [297]:
# Создание и обучение линейной регрессионной модели
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [298]:
# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)

In [299]:
# Оценка точности модели (RMSE)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

RMSE: 4.497004600511498e-09


In [302]:
def get_top_3_destinations(origin):
    # Получаем индексы всех строк с указанным аэропортом вылета
    rows = X[X['ORIGIN_AIRPORT'] == origin]
    
    # Прогнозируем задержку для этих строк
    predictions = model.predict(rows)
    
    # Формируем словарь, где ключ - индекс строки, значение - предсказанная задержка
    delay_dict = dict(zip(rows.index, predictions))
    
    # Сортируем аэропорты назначения по возрастанию задержки
    sorted_delays = sorted(delay_dict.items(), key=lambda x: x[1])
    
    # Проверяем, что среди первых трёх аэропортов нет повторений
    unique_destinations = []
    seen_destinations = set()
    
    for idx, _ in sorted_delays:
        dest = X.at[idx, 'DESTINATION_AIRPORT']
        if dest not in seen_destinations:
            unique_destinations.append(dest)
            seen_destinations.add(dest)
        
        if len(unique_destinations) >= 3:
            break
            
    if len(unique_destinations) < 3:
        print('Невозможно построить топ-3 аэропорта прибытия для этого аэропорта отправления, так как их меньше 3.')
        #return unique_destinations#raise ValueError(f"Not enough unique destinations found for origin airport '{origin}'.")
    
    return unique_destinations

In [304]:
#Я допишу обратное кодирование, будут вводиться и выводиться названия аэропортов или их IATA-коды

origin_airport = 50
top_3 = get_top_3_destinations(origin_airport)
print(f"Топ аэропортов назначения для вылета из {origin_airport}: {top_3}")

Топ аэропортов назначения для вылета из 50: [304, 267, 240]
